## makemore - MLP

### Implementing [Bengio, et al. 2003: A Neural Probabilistic Language Model](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

This implementation introduces:
- more sophisticated embeddings
- hyperparameters
- learning rate tuning
- train/test/dev set split
- under- and over-fitting

#### Exercises:
- [ ] E01: Tune the hyperparameters of the training to beat a validation loss of 2.2
- [ ] E02: (1) What is the loss you'd get if the predicted probabilities at initialization were perfectly uniform? What loss do we achieve? (2) Can you tune the initialization to get a starting loss that is much more similar to (1)?
- [ ] E03: Read the Bengio et al 2003 paper (link above), implement and try any idea from the paper. Did it work?

### Exercise: 